- [Spark programming guide](http://spark.apache.org/docs/latest/programming-guide.html#accumulators)
- [Databricks Spark Knowledgebase](https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/)

## Setup

In [ ]:
docker-machine start
(docker-machine env)

In [ ]:
@FOR /f "tokens=*" %i IN ('docker-machine env') DO @%i

In [ ]:
docker run -it bigdatauniversity/spark bash

In [ ]:
/etc/bootstrap.sh

In [ ]:
spark-shell

## Restart

In [ ]:
#If you exit from Docker Container, you can always restart and attach to it later by running the below:
docker start  bdu_spark2 
docker attach bdu_spark2

#Start a new command in a running container
docker exec -it bdu_spark2 <command>

## Use zeppeling

In [ ]:
https://hub.docker.com/r/bigdatauniversity/spark2/

## Initializing Spark - Scala

In [ ]:
SparkConf contains information about application

## Passing functions to Spark

- Anonymous function syntax:
    (x: Int) => x + 1
- Static methods
    object MyFunctions {
        def func1 (s: String): String = (...)
    }
- Passing by reference
    val field = "Hello"
    

## Create an RDD

In [ ]:
RDD = Resilient Distributed Dataset

In [ ]:
val data = 1 to 1000

In [ ]:
val distData = sc.parallelize(data)

### Value RDD to Pair RDD

In [ ]:
val byStartTerminal = trips.keyBy(_.startStation) //keyBy

## Print out first 5 lines

In [ ]:
grouped.take(5).foreach(println)

## Cache

Use the cache when you want to do repeat actions after transformations. [see this explanation](http://stackoverflow.com/questions/28981359/why-do-we-need-to-call-cache-or-persist-on-a-rdd)
- cache is a special case of persist:
    cache() == persist(StorageLevel.MEMORY_ONLY)

## Transformations

- map(func)
   - Return new dataset formed by passing each element of source through func
- filter(func)
   - Return new dataset by selecting those elements of the source on which func returns true
- flatMap(func)
   - Similar to map, but func can return a Seq rather than a single item
- join
- reduceByKey


## Actions

- collect()
- count()
- first()
- take(n)
- foreach(func)

## Persistence

- persist()
- cache()

## Shared variables

### Broadcast variables

Used when you have a large dataset that you want to use acress all the worker nodes. Instead of sending all data, just send the variable

In [ ]:
val broadcastVar = sc.broadcast(Array(1,2,3))

In [ ]:
=> mapping before shuffling

### Accumulators

Run **counters** and **sum** efficiently in parallel

# Scheduler

Fair scheduler appropriate for multi-user

In [ ]:
spark.scheduler.mode = FAIR // (default FIFO (first in first out))

Pools can have their own scheduler
- Useful for priority queries
- Zeppelin gives each user a pool

# Troubleshoot "Task not serializable"¶

In [ ]:
class MyHelper extends Serializable { // >>extends Serializable<< does the trick
    def doSomething (input:String):String = inner.doStuff(input)
}
val helper = new MyHelper()
val output = input1.map(helper.doSomething(_))

# Serialization

- Java
- Kryo (more efficient, more configuration)
- pickle (Python)

# Examples

### SQL Example

#### Infer schema using reflection

In [ ]:
case class Person(name: String, age: Int)

In [ ]:
val people = sc.textFile("examples/people.txt")
.map(_.split(","))
.map(p => Person(p(0), p(1).trim.toInt))

In [ ]:
people.registerTempTable("people")

In [ ]:
val teenagers = sqlContext.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")

In [ ]:
teenagers.map(t => "Name: " + t(0)).collect().foreach(println)

#### Programmatic interface

In [ ]:
val people = sc.textFile(..)

In [ ]:
val schemaString = "name age"

In [ ]:
val schema = StructType( schemaString.split(" ")
.map(fieldName => StructField(fieldName, StringType, true)))

In [ ]:
val rowRDD = people.map(_.split(",")).map(p => Row(p(0), p(1).trim))
val peopleSchemaRDD = sqlContext.applySchema(rowRDD, schema)

In [ ]:
peopleSchemaRDD.registerTempTable("people")

In [ ]:
val results = sqlContext.sql("SELECT name FROM people")
results.map(t => "Name: " + t(0)).collect().foreach(println)

### Example - avg per key

In [ ]:
// BAD way to do: groupByKey()
val keyedRdd: RDD[(String, Int)] = 
keyedRdd.groupByKey().mapValues(I => I.sum / I.size)

// GOOD way to do: aggregateByKey()
val results = keyedRdd.aggregateByKey((0,0))( // (0,0) is initial zero accumulator
(acc, value) => (acc._1 + value, acc._2 + 1),
(acc, acc2) => (acc1._1 + acc2._1, acc1._2 + acc2._2)
)
results.mapValues(i => i._1/i._2)

## Partitions

- mapValues: Always use mapValues when possible because with map partition gets flöten
- foreachPartition: one partition at a time

### Breadcast variables

# Questions

In [ ]:
What does _ in .. mean?
input.map(_.split(",")).count()